In [24]:
from pathlib import Path
from ultralytics import YOLO
import cv2
import numpy as np

# Load model
model = YOLO('/home/andes/skripsi/Nila-Counting/runs/train/E200B4/weights/best.pt')  # Ganti dengan path ke model best.pt Anda

p = Path("./Test-Vid").glob("**/*")
list_video = list(p)
actual = []
total_bibit = []

for video in list_video:
    filename = str(video).split("/")[1]
    number_seed = int(filename.split(".")[0])
    
    actual.append(number_seed)
    
    cap = cv2.VideoCapture(video)

    # Mendapatkan ukuran frame untuk output video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Variabel untuk menghitung total jumlah bibit dan jumlah frame
    total_count = 0
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Melakukan inferensi pada setiap frame
        results = model.predict(frame, save=False)  # Prediksi tanpa menyimpan otomatis
        
        # Hitung jumlah deteksi (jumlah bounding box yang ditemukan)
        count = 0  # Inisialisasi perhitungan jumlah bibit ikan
        for result in results:
            count = len(result.boxes)  # Hitung jumlah box deteksi

            # Gambar bounding box tanpa label
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Bounding box hijau

        # Tambahkan ke total_count dan tambahkan frame_count
        total_count += count
        frame_count += 1

        # Tampilkan jumlah bibit ikan yang terdeteksi di layar
        text = f'Total number of seeds detected: {count}'
        
        # Lokasi teks
        org = (20, 40)
        
        # Tambahkan shadow (teks hitam di belakang)
        cv2.putText(frame, text, (org[0] + 2, org[1] + 2), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 4)  # Bayangan hitam, tebal 4 px
        
        # Tambahkan teks putih di atas shadow (Bold dengan ketebalan lebih)
        cv2.putText(frame, text, org, cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)  # Teks merah, tebal 3 px



        # Tampilkan hasil secara langsung (opsional)
        cv2.imshow('YOLOv10 Inference', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Hitung rata-rata jumlah bibit per frame
    average_count = total_count / frame_count if frame_count > 0 else 0

    total_bibit.append(average_count)

    # Melepaskan resource
    cap.release()

    cv2.destroyAllWindows()
    


0: 640x544 92 BibitNilas, 8.1ms
Speed: 2.1ms preprocess, 8.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 93 BibitNilas, 25.9ms
Speed: 3.1ms preprocess, 25.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 91 BibitNilas, 11.2ms
Speed: 3.8ms preprocess, 11.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 95 BibitNilas, 36.3ms
Speed: 2.3ms preprocess, 36.3ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 94 BibitNilas, 9.2ms
Speed: 2.8ms preprocess, 9.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 94 BibitNilas, 10.5ms
Speed: 2.5ms preprocess, 10.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 94 BibitNilas, 11.4ms
Speed: 3.5ms preprocess, 11.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 93 BibitNilas, 12.8ms
Speed: 3.0ms preprocess, 12.8ms inference, 1.6ms post

In [25]:
from sklearn.metrics import root_mean_squared_error

print(root_mean_squared_error(actual, total_bibit))

4.990775653611073


In [21]:
from sklearn.metrics import mean_squared_error
import math

# Assuming `actual` and `total_bibit` are the lists of ground truth and predicted values
rmse = math.sqrt(mean_squared_error(actual, total_bibit))
print(f"RMSE: {rmse}")


RMSE: 2.8378160009616944
